# 主要是測試各種功能用的，自行增加區塊來測試(記得加註解)

## torch測試及可用顯存測試

In [ ]:
import torch
from psutil import virtual_memory
print(torch.cuda.is_available())
GPUmem = torch.cuda.mem_get_info()
mem = GPUmem[1] /1024 / 1024 / 1024
mem += virtual_memory().total /1024 / 1024 / 1024 / 2
mem

## 功能檔案分離測試

In [ ]:
import WebDisplay
data = {
    'name': 'example',
    'value': 42,
    'status': 'active'
}
web_display = WebDisplay.WebDisplay(data)
web_display.print_data()

## 下載YouTube影片及聲音測試

In [ ]:
from bs4 import BeautifulSoup
from pytubefix import YouTube
yt = YouTube('https://music.youtube.com/watch?v=NycFr6D6DSw&si=przy_9oxIVQs5HI9')
print('download...')
yt.streams.get_audio_only().download(output_path='download/')   #下載音訊
print('ok!')

In [ ]:
acc = yt.streams.get_audio_only()
acc.download(output_path='download/')   #下載音訊

### 加入手動輸入連結

In [ ]:
url = input()

In [ ]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation
import os
vedio = YouTubeDownload.download_youtube_video(url)
audio = YouTubeDownload.download_youtube_audio(url)
separation_audio = MusicSeparation.run_separation(audio)
merge = YouTubeDownload.merge_video_audio(vedio, separation_audio[0])
print(vedio, audio , merge, separation_audio)
#os.system(merge)
separation_audio

In [ ]:
os.system(merge)
audio

In [ ]:
#測試用，記得刪
from MusicSeparation import MusicSeparation
from YouTubeDownload import YouTubeDownload
separation_audio = ['h:\\NFU\\score\\score/output\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]-other.wav',
 'h:\\NFU\\score\\score/output\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]\\光輝歲月 - Beyond ( 黃家駒, 黃貫中, 黃家強, 葉世榮 )[ lyrics ]-vocals.wav']

### 整合ASR辨識

In [ ]:
ASR_result = MusicSeparation.run_ASR(separation_audio[1])

In [ ]:
YouTubeDownload.download_captions_by_language_code(video_url=url,language_code=ASR_result[1])

In [ ]:
from YouTubeDownload import YouTubeDownload
YouTubeDownload.download_captions_by_language_code(video_url=url,language_code='ja')

## NeMo測試(確認無用)

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr

print(nemo.__version__)


## Whisper字幕辨識測試

In [ ]:
import whisper
whisper.available_models()

In [ ]:
import os
import torch
import tkinter as tk
from tkinter import filedialog
import whisper

#建立Tk主視窗，但隱藏
root = tk.Tk()
root.withdraw()
root.attributes('-topmost', True)  #對話框出現在最前方

#選擇音訊檔案
current_dir = os.getcwd()
file_path = filedialog.askopenfilename(
    title="請選擇音訊檔案",
    initialdir=current_dir,
    filetypes=[("音訊檔案", "*.wav *.mp3 *.m4a *.flac"), ("所有檔案", "*.*")]
)

if not file_path:
    print("未選擇檔案")
    exit()

print("選擇的檔案:", file_path)

#載入Whisper模型
if(torch.cuda.is_available()):
    model = whisper.load_model("large-v3", device="cuda")
else:
    model = whisper.load_model("large-v3", device="cpu")

#轉錄音訊，Whisper會自動偵測語言
result = model.transcribe(file_path)
print("偵測語言:", result["language"])

#輸出資料夾:與音訊檔同一層
audio_dir = os.path.dirname(file_path)

#取得音訊檔名稱(不含副檔名)
filename = os.path.splitext(os.path.basename(file_path))[0]

#---SRT---
with open(os.path.join(audio_dir, f"{filename}.srt"), "w", encoding="utf-8") as f:
    for i, seg in enumerate(result["segments"], start=1):
        text = seg['text'].strip()
        if not text:
            continue
        f.write(f"{i}\n")
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02},{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02},{int((end%1)*1000):03}"
        f.write(f"{start_time} --> {end_time}\n")
        f.write(f"{text}\n\n")

#---VTT---
with open(os.path.join(audio_dir, f"{filename}.vtt"), "w", encoding="utf-8") as f:
    f.write("WEBVTT\n\n")
    for seg in result["segments"]:
        text = seg['text'].strip()
        if not text:
            continue
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02}.{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02}.{int((end%1)*1000):03}"
        f.write(f"{start_time} --> {end_time}\n")
        f.write(f"{text}\n\n")

#---TXT---
with open(os.path.join(audio_dir, f"{filename}.txt"), "w", encoding="utf-8") as f:
    for seg in result["segments"]:
        text = seg['text'].strip()
        if not text:
            continue
        start = seg["start"]
        end = seg["end"]
        start_time = f"{int(start//3600):02}:{int((start%3600)//60):02}:{int(start%60):02}.{int((start%1)*1000):03}"
        end_time = f"{int(end//3600):02}:{int((end%3600)//60):02}:{int(end%60):02}.{int((end%1)*1000):03}"
        f.write(f"[{start_time} --> {end_time}] {text}\n")

print(f"已輸出至資料夾:{audio_dir}")


### 分離功能至MusicSeparation.py內測試

In [ ]:
from MusicSeparation import MusicSeparation
#MusicSeparation.run_ASR(r"H:/NFU/score/score/output/Immortals/Immortals-vocals.wav")    #若無傳入值則由使用者手動選取
MusicSeparation.run_ASR()

## Whisper微調測試(官方範例)

In [ ]:
from datasets import load_dataset, DatasetDict
import pandas as pd

common_voice = DatasetDict()

#載入文件範例("mozilla-foundation/common_voice_11_0"是線上的資料，要自己改)
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation", trust_remote_code=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test", trust_remote_code=True)

#common_voice = pd.read_csv("processed_songs/metadata_all.csv")
common_voice["train"]["audio"]

### 這裡開始

In [ ]:
import pandas as pd
from datasets import load_dataset, Audio ,DatasetDict

#讀取原始CSV
df = pd.read_csv("processed_songs/metadata_all.csv")

#改欄位名稱符合Hugging Face標準
df = df.rename(columns={"file": "audio", "text": "sentence"})
df = df[["audio","sentence"]]

#存成新的CSV
df.to_csv("processed_songs/metadata_hf.csv", index=False)

#用Hugging Face載入新的CSV
dataset = load_dataset(
    "csv",
    data_files={"train": "processed_songs/metadata_hf.csv"}
)

#直接在cast_column中指定目標採樣率
#這會告訴datasets在讀取音檔時，自動解碼並重採樣到16000Hz
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
common_voice = dataset

#---開始偵錯---

#1.先把第一筆音訊資料取出來，存成一個變數
first_audio_data = dataset["train"][0]["audio"]

#2.印出這個變數的「型別」，看看它到底是什麼
print("變數的型別:", type(first_audio_data))
print("-" * 20)

#3.嘗試存取它裡面的key(例如"sampling_rate"或"array")
#這個動作會「強制」datasets 去執行解碼和重採樣
print("音訊的採樣率:", first_audio_data["sampling_rate"])
print("音訊波形陣列 (前5個點):", first_audio_data["array"][:5])
print("-" * 20)

#4.再次印出整個變數
print("強制解碼後，完整的變數內容:")
print(first_audio_data)

Generating train split: 0 examples [00:00, ? examples/s]

變數的型別: <class 'datasets.features._torchcodec.AudioDecoder'>
--------------------
音訊的採樣率: 16000
音訊波形陣列 (前5個點): [-0.00033569  0.00146484 -0.00140381  0.00088501  0.00115967]
--------------------
強制解碼後，完整的變數內容:


In [2]:
#特徵提取器(預處理音檔)_選擇模型
#模型輸出後處理為文字格式的分詞器
#language要自己改
from transformers import WhisperFeatureExtractor, WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Chinese", task="transcribe")


In [3]:
input_str = common_voice["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 塞納河
Decoded w/ special:    <|startoftranscript|><|zh|><|transcribe|><|notimestamps|>塞納河<|endoftext|>
Decoded w/out special: 塞納河
Are equal:             True


In [4]:
from transformers import WhisperProcessor
"""輸出格式範例(採樣率48K)
{'audio': {'path': '/home/sanchit_huggingface_co/.cache/huggingface/datasets/downloads/extracted/607848c7e74a89a3b5225c0fa5ffb9470e39b7f11112db614962076a847f3abf/cv-corpus-11.0-2022-09-21/hi/clips/common_voice_hi_25998259.mp3', 
           'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.6724887e-07,
       1.5334779e-06, 1.0415988e-06], dtype=float32), 
           'sampling_rate': 48000},
 'sentence': 'खीर की मिठास पर गरमाई बिहार की सियासत, कुशवाहा ने दी सफाई'}
"""
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="zh", task="transcribe")

#print(common_voice["train"][0])

In [5]:
#from datasets import Audio
"""輸出格式範例(下採樣至16K)
{'audio': {'path': '/home/sanchit_huggingface_co/.cache/huggingface/datasets/downloads/extracted/607848c7e74a89a3b5225c0fa5ffb9470e39b7f11112db614962076a847f3abf/cv-corpus-11.0-2022-09-21/hi/clips/common_voice_hi_25998259.mp3', 
           'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
       -3.4206650e-07,  3.2979898e-07,  1.0042874e-06], dtype=float32),
           'sampling_rate': 16000},
 'sentence': 'खीर की मिठास पर गरमाई बिहार की सियासत, कुशवाहा ने दी सफाई'}
"""
#common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

print(common_voice["train"][0])

{'audio': <datasets.features._torchcodec.AudioDecoder object at 0x000001C89B801090>, 'sentence': '塞納河'}


In [ ]:
#初始化processor(包含feature_extractor + tokenizer)
def prepare_dataset(batch):
    from transformers import WhisperProcessor
    import torch
    processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="zh", task="transcribe")
    audio = batch["audio"]
    batch["input_features"] = processor.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    batch["labels"] = processor.tokenizer(batch["sentence"]).input_ids
    batch["labels"] = torch.tensor(batch["labels"], dtype=torch.long)
    return batch

common_voice = common_voice.map(
    prepare_dataset,
    remove_columns=common_voice["train"].column_names,
    num_proc=14
)


Map (num_proc=14):   0%|          | 0/3518 [00:00<?, ? examples/s]

In [8]:
#定義數據收集器
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        #split inputs and labels since they have to be of different lengths and need different padding methods
        #first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        #get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        #pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        #replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        #if bos token is appended in previous tokenization step,
        #cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [9]:
#初始化數據整理器
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=processor.tokenizer.bos_token_id,  #Whisper的decoder開始token
)

In [ ]:
import evaluate

#wer代表詞錯誤率
metric = evaluate.load("wer")

#錯誤率評估函式
def compute_metrics(pred):
    #取出預測結果
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    #如果predictions是tuple(logits,...)
    if isinstance(pred_ids, tuple):
        pred_ids = pred_ids[0]

    #如果是logits，取argmax
    if pred_ids.ndim == 3:
        pred_ids = pred_ids.argmax(-1)

    #把-100換成pad_token_id
    import numpy as np
    label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)

    #解碼成文字
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    #計算WER
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}


In [11]:
#載入預訓練checkpoint
from transformers import WhisperForConditionalGeneration

#模型自己改
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [16]:
from transformers import Seq2SeqTrainingArguments

#如果不想將模型checkpoint上傳到Hub，設定push_to_hub=False
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-zh",  #change to a repo name of your choice
    per_device_train_batch_size=8, #如果出現OOM(顯存超載)錯誤將此減半
    gradient_accumulation_steps=2,  #並將此乘2
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=False,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [ ]:
#從訓練資料集中抓出測試資料
common_voice = common_voice["train"].train_test_split(test_size=0.1)

In [17]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

C:\Users\a0987\AppData\Local\Temp\ipykernel_65288\240849381.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [18]:
#訓練模型
trainer.train()

Step,Training Loss,Validation Loss,Wer
1000,0.092900,0.752077,58.659218
2000,0.006100,0.833569,58.100559
3000,0.000100,0.869361,55.307263
4000,0.000100,0.883776,56.424581
5000,0.000100,0.889415,56.983240


Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\a0987\anaconda3\envs\aligner\Lib\site-packages\transformers\modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. Yo

TrainOutput(global_step=5000, training_loss=0.4101779097528663, metrics={'train_runtime': 14601.5638, 'train_samples_per_second': 5.479, 'train_steps_per_second': 0.342, 'total_flos': 2.3072402755584e+19, 'train_loss': 0.4101779097528663, 'epoch': 25.252525252525253})

In [19]:
#設定上傳模型數值
kwargs = {
    "dataset_tags": "Zzzkay1/song", #標籤，記得改
    "dataset": "Chinese songs * 14",  #a 'pretty' name for the training dataset (翻譯)給資料集設定一個你喜歡的名子
    "dataset_args": "config: zh, split: test",
    "language": "zh", #語言自己改
    "model_name": "Whisper Small zh - Song train",  #a 'pretty' name for your model (翻譯)給模型一個好名子
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

### 上傳

In [ ]:
#儲存模型pipeline使用的資料
processor.save_pretrained("./whisper-small-zh")
#上傳模型
trainer.push_to_hub(**kwargs)

CommitInfo(commit_url='https://huggingface.co/Zzzkay1/whisper-small-zh/commit/5da4daebedba65014b675cca7283b4094da1cd88', commit_message='End of training', commit_description='', oid='5da4daebedba65014b675cca7283b4094da1cd88', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Zzzkay1/whisper-small-zh', endpoint='https://huggingface.co', repo_type='model', repo_id='Zzzkay1/whisper-small-zh'), pr_revision=None, pr_num=None)

### 訓練完後如何使用

In [8]:
from transformers import pipeline
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation
import os
import gradio as gr
if True:
    tempmodel = "Zzzkay1/whisper-small-zh"
else:
    tempmodel = "openai/whisper-small"
pipe = pipeline(
    "automatic-speech-recognition",
    model=tempmodel,#改成你自己的模型tag
    return_timestamps=True,
)

def fun(url1):
    audio = YouTubeDownload.download_youtube_audio(url1)
    video = YouTubeDownload.download_youtube_video(url1)
    separtionMusic = MusicSeparation.run_separation(audio)
    merge = YouTubeDownload.merge_video_audio(video_path=video, audio_path=separtionMusic[0])

    #去掉多餘的引號&替換不合法字元
    merge = merge.strip('"').strip("'")
    safe_name = os.path.basename(merge).replace("／", "_").replace("/", "_").replace("\\", "_")
    
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, safe_name)

    if merge != out_path:
        try:
            os.replace(merge, out_path)  #搬移檔案
        except Exception as e:
            print("搬檔失敗:", e)
            out_path = merge  #如果搬失敗，就還是用原本路徑

    if not os.path.exists(out_path):
        return f"檔案不存在: {out_path}"

    results = pipe(separtionMusic[1])["text"]
    #return out_path  #給gr.Video()
    with open("OutputFile.txt","w",encoding="utf-8") as f:
        f.write("\n".join(result))
    return results
"""
沒用
def transcribe(audio):
    text = pipe(audio)["text"]
    return text
"""

iface = gr.Interface(
    fn=fun,
    inputs=gr.Textbox(lines=1, placeholder="請輸入YouTube連結"), #輸入文字區
    outputs="text", #輸出為文字
    title="Whisper Small zh", #標題自訂
    description="Realtime demo for zh speech recognition using a fine-tuned Whisper small model.", #描述自訂
)
    
iface.launch() #啟動

Device set to use cuda:0


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


已儲存：h:\NFU\score\score/output\周杰倫 Jay Chou【蘭亭序 Lan-Ting-Xu】-Official Music Video\周杰倫 Jay Chou【蘭亭序 Lan-Ting-Xu】-Official Music Video-vocals.wav
已儲存：h:\NFU\score\score/output\周杰倫 Jay Chou【蘭亭序 Lan-Ting-Xu】-Official Music Video\周杰倫 Jay Chou【蘭亭序 Lan-Ting-Xu】-Official Music Video-other.wav


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


## 人聲分離顯存佔用最佳化測試

In [ ]:
import torch
import torchaudio
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB
from torchaudio.models import hdemucs_high
from psutil import virtual_memory
import subprocess
import os
import soundfile as sf
from torchaudio.transforms import Fade

def _separate_sources(model, mix, sample_rate, device, segment=15.0, overlap=0.1):
    """
    分段執行音源分離，避免一次丟整首造成爆顯存
    """
    batch, channels, length = mix.shape

    chunk_len = int(sample_rate * segment * (1 + overlap))
    start = 0
    end = chunk_len
    overlap_frames = int(overlap * sample_rate)
    fade = Fade(fade_in_len=0, fade_out_len=overlap_frames, fade_shape="linear")

    final = torch.zeros(batch, len(model.sources), channels, length, device=device)

    while start < length - overlap_frames:
        chunk = mix[:, :, start:end]
        with torch.no_grad():
            out = model.forward(chunk)
        out = fade(out)
        final[:, :, :, start:end] += out

        if start == 0:
            fade.fade_in_len = overlap_frames
            start += chunk_len - overlap_frames
        else:
            start += chunk_len
        end += chunk_len
        if end >= length:
            fade.fade_out_len = 0

    return final

def run_separation(audio_path, segment = 15.0, use_cuda = True):
    try:
        bundle = HDEMUCS_HIGH_MUSDB
        model = bundle.get_model()
    except Exception as e:
        return e

    try:
        GPUmem = torch.cuda.mem_get_info()[1] / 1024 / 1024 / 1024
        SYSmem = virtual_memory().total / 1024 / 1024 / 1024 / 2
        if torch.cuda.is_available() and GPUmem > 2 and use_cuda:
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        model = model.to(device)

        input_file = audio_path.strip('"')
        if not os.path.exists(input_file):
            return f"找不到輸入檔案：{input_file}"

        ext = os.path.splitext(input_file)[1].lower()
        if ext == '.wav':
            waveform, sample_rate = torchaudio.load(input_file)
        else:
            temp_wav = "temp_convert.wav"
            try:
                subprocess.run([
                    "ffmpeg", "-y", "-i", input_file, "-acodec", "pcm_s16le", "-ar", "44100", temp_wav
                ], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                data, sample_rate = sf.read(temp_wav, dtype='float32')
                if data.ndim == 1:
                    data = data[:, None]
                waveform = torch.from_numpy(data.T)
            except subprocess.CalledProcessError as ffmpeg_err:
                return ffmpeg_err
            finally:
                if os.path.exists(temp_wav):
                    os.remove(temp_wav)

        if sample_rate != bundle.sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=bundle.sample_rate)
            waveform = resampler(waveform)
            sample_rate = bundle.sample_rate

        waveform = waveform.to(device)

        #🔹 使用分段方式做分離（segment=15s, overlap=0.1）
        sources = _separate_sources(
            model=model,
            mix=waveform.unsqueeze(0),
            segment = segment,
            sample_rate=sample_rate,
            device=device
        )[0]

        base_name = os.path.splitext(os.path.basename(input_file))[0]
        output_dir = os.path.join(os.getcwd(), "score/output", base_name)
        os.makedirs(output_dir, exist_ok=True)

        other_mix = sources[0] + sources[1] + sources[2]
        vocals = sources[3]

        torchaudio.save(os.path.join(output_dir, base_name + '-vocals.wav'), vocals.cpu(), sample_rate)
        torchaudio.save(os.path.join(output_dir, base_name + '-other.wav'), other_mix.cpu(), sample_rate)

        print(f"已儲存：{os.path.join(output_dir, base_name + '-vocals.wav')}")
        print(f"已儲存：{os.path.join(output_dir, base_name + '-other.wav')}")

        del sources
        del waveform
        torch.cuda.empty_cache()

        output = [
            f"{os.path.join(output_dir, base_name + '-other.wav')}",
            f"{os.path.join(output_dir, base_name + '-vocals.wav')}"
        ]
        return output

    except Exception as e:
        try:
            if 'sources' in locals():
                del sources
            if 'waveform' in locals():
                del waveform
        except:
            pass
        torch.cuda.empty_cache()
        return e


In [ ]:
run_separation(r"H:\NFU\score\download\MIMI - サイエンス (feat.重音テトSV).m4a", segment=15, use_cuda=True)

## 網頁測試(暫定)

In [ ]:
import gradio as gr
import os
from MusicSeparation import MusicSeparation
from YouTubeDownload import YouTubeDownload

def fun(url1):
    audio = YouTubeDownload.download_youtube_audio(url1)
    video = YouTubeDownload.download_youtube_video(url1)
    separtionMusic = MusicSeparation.run_separation(audio)
    merge = YouTubeDownload.merge_video_audio(video_path=video, audio_path=separtionMusic[0])

    #去掉多餘的引號 & 替換不合法字元
    merge = merge.strip('"').strip("'")
    safe_name = os.path.basename(merge).replace("／", "_").replace("/", "_").replace("\\", "_")
    
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, safe_name)

    if merge != out_path:
        try:
            os.replace(merge, out_path)  #搬移檔案
        except Exception as e:
            print("搬檔失敗:", e)
            out_path = merge  #如果搬失敗，就還是用原本路徑

    if not os.path.exists(out_path):
        return f"檔案不存在: {out_path}"

    return out_path  #給 gr.Video()


iface = gr.Interface(
    fn=fun,
    inputs=gr.Textbox(lines=1, placeholder="請輸入YouTube連結"),
    outputs=gr.Video(),
    title="哈哈標題",
    description="輸入Youtube連結自動下載並分離人聲及音樂",
)

iface.launch()